## LangServe Server Setup

This notebook is a playground for those interested in developing interactive web applications using LangChain and [**LangServe**](https://python.langchain.com/docs/langserve). The aim is to provide a minimal-code example to illustrate the potential of LangChain in web application contexts.

This section provides a walkthrough for setting up a simple API server using LangChain's Runnable interfaces with FastAPI. The example demonstrates how to integrate a LangChain model, such as `ChatNVIDIA`, to create and distribute accessible API routes. Using this, you will be able to supply functionality to the frontend service's [**`frontend_server.py`**](frontend/frontend_server.py) session, which strongly expects:
- A simple endpoint named `:9012/basic_chat` for the basic chatbot, exemplified below.
- A pair of endpoints named `:9012/retriever` and `:9012/generator` for the RAG chatbot.
- All three for the **Evaluate** utility, which will be required for the final assessment. *More on that later!*

**IMPORTANT NOTES:**
- Make sure to click the square ( $\square$ ) button twice to shut down an active FastAPI cell. The first time might fall through or trigger a try-catch routine on an asynchronous process.
- If it still doesn't work, do a hard restart on this notebook by using **Kernel -> Restart Kernel**.
- When a FastAPI server is running in your cell, expect the process to block up this notebook. Other notebooks should not be impacted by this. 

In [ ]:
%%writefile server_app.py
# https://python.langchain.com/docs/langserve#server
from fastapi import FastAPI
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings
from langserve import add_routes

## May be useful later
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.runnables import RunnableLambda, RunnableBranch, RunnablePassthrough
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_community.document_transformers import LongContextReorder
from functools import partial
from operator import itemgetter

from langchain_community.vectorstores import FAISS

## TODO: Make sure to pick your LLM and do your prompt engineering as necessary for the final assessment
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")
llm = instruct_llm | StrOutputParser()

## Prompt
chat_prompt = ChatPromptTemplate.from_messages([("system",
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked you a question: {input}\n\n"
    " The following information may be useful for your response: "
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational)"
), ('user', '{input}')])

docstore = FAISS.load_local("docstore_index", embedder, allow_dangerous_deserialization=True)
docs = list(docstore.docstore._dict.values())

app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple api server using Langchain's Runnable interfaces",
)

## PRE-ASSESSMENT: Run as-is and see the basic chain in action

add_routes(
    app,
    instruct_llm,
    path="/basic_chat",
)

## ASSESSMENT TODO: Implement these components as appropriate

add_routes(
    app,
    docstore.as_retriever(),
    path="/retriever",
)

add_routes(
    app,
    chat_prompt | llm ,
    path="/generator",
)

## Might be encountered if this were for a standalone python file...
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=9012)